In [1]:
from cobraCoach import CobraCoach
import pathlib
import numpy as np
import logging
import engineer as eng

In [2]:
mod = 'SC09'
cc = CobraCoach('fpga', loadModel=False)
cc.loadModel(version=mod, moduleVersion=15)
eng.setCobraCoach(cc)

/home/cwen/mhs/devel/ics_cobraCharmer/python/ics/cobraCharmer/utils/butler.py:178: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yamlFile)
2020-08-21 14:27:49.696Z butler           30 butler.py:58 newRun: /data/MCS/20200821_012
2020-08-21 14:27:49.698Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-21 14:27:49.734Z camera           20 asrdCam.py:17 asrd...


In [21]:
eng.mmTheta = np.load('/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest/mmThetaSlow2.npy')
eng.mmPhi = np.load('/home/cwen/mhs/devel/ics_cobraCharmer/procedures/moduleTest/mmPhiSlow2.npy')

In [ ]:
# update center positions from home, not accurately
eng.convertXML(f'{mod}.xml')

In [ ]:
cc.showStatus()

In [36]:
# enable motor scaling and set parameters
cc.setScaling(True, thetaScaleFactor=4.0, phiScaleFactor=4.0, minThetaSteps=1, minPhiSteps=1)

In [4]:
cc.setScaling(True, thetaScaleFactor=2.0, phiScaleFactor=2.0, minThetaSteps=10, minPhiSteps=10)

In [ ]:
cc.setScaling(True, thetaScaleFactor=2.0, phiScaleFactor=2.0, minThetaSteps=200, minPhiSteps=200)

In [ ]:
# reset motor scaling
cc.pfi.resetMotorScaling()

In [8]:
# switch to phi mode
eng.setPhiMode()

In [9]:
# generate a quick motormap and set geometry
eng.makePhiMotorMaps(f'{mod}_phi_400.xml', steps=200, totalSteps=6000, repeat=1, fast=False)

2020-08-21 14:56:02.292Z butler           30 butler.py:58 newRun: /data/MCS/20200821_015
2020-08-21 14:56:02.295Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-21 14:56:02.296Z camera           20 asrdCam.py:17 asrd...


Camera close!


2020-08-21 14:56:02.988Z cobraCoach       20 cobraCoach.py:944 phi home -5000 steps
2020-08-21 14:56:02.989Z pfi              30 pfi.py:546 clipping (18, 'theta', 'cw') ontime to 0.12 and scale 1.15 to 1.0


Camera close!
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103149829
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103118685
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 14:56:27.892Z cobraCoach       20 cobraCoach.py:956 1/1 phi forward to 200


...

getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 14:59:27.772Z cobraCoach       20 cobraCoach.py:991 1/1 phi backward to 6000


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 14:59:30.777Z cobraCoach       20 cobraCoach.py:1015 1/1 phi reverse -5000 steps to limit
2020-08-21 14:59:43.659Z pfiDesign        20 pfiDesign.py:807 wrote pfiDesign file for 57 cobras and name=None to /data/MCS/20200821_015/output/SC09_phi_400.xml


(PosixPath('/data/MCS/20200821_015'), array([], dtype=int64))

In [10]:
# set phi geometry from previous run
cc.setPhiGeometryFromRun('/data/MCS/20200821_015', onlyIfClear=False)

In [ ]:
# move phi to home position
cc.moveToHome(cc.goodCobras, phiEnable=True)

In [ ]:
# phi on time search
eng.phiOnTimeSearch(f'{mod}_phi_search.xml', iteration=3, repeat=1)

In [ ]:
# generate new motormaps
#eng.makePhiMotorMaps(f'{mod}_phi800.xml', steps=800, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi500.xml', steps=500, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi200.xml', steps=200, totalSteps=8000, repeat=3)
eng.makePhiMotorMaps(f'{mod}_phi100.xml', steps=100, totalSteps=8000, repeat=1)
#eng.makePhiMotorMaps(f'{mod}_phi50.xml', steps=50, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi20.xml', steps=20, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi10.xml', steps=10, totalSteps=8000, repeat=3)
#eng.makePhiMotorMaps(f'{mod}_phi5.xml', steps=5, totalSteps=6000, repeat=3)

In [ ]:
eng.makePhiMotorMaps(f'{mod}_phi_fast.xml', steps=50, totalSteps=6000, repeat=5, fast=True)

In [ ]:
eng.makePhiMotorMaps(f'{mod}_phi_test.xml', steps=25, totalSteps=10000, repeat=3, fast=True, phiOnTime=phiOt)

In [ ]:
# move phi by angles
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.deg2rad(165), relative=False, local=True, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# move by a angle
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.deg2rad(90), relative=False, local=True, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# convergence test
moves = eng.phiConvergenceTest(cc.goodIdx, margin=15.0, runs=51, tries=8, fast=False, tolerance=0.1)

In [ ]:
moves = eng.phiConvergenceTest2(cc.goodIdx, margin=15.0, runs=51, tries=8, threshold=0.2, tolerance=0.0015)

In [ ]:
# move phi to 60 degree for theta test
dataPath, diffAngles, moves = eng.movePhiAngles(cc.goodIdx, np.pi/3, relative=False, local=False, tolerance=0.002, tries=6, fast=False, newDir=True)

In [ ]:
# phi on-time scan
dataPath, ontimes, angles, speeds = eng.phiOntimeScan(speed=np.deg2rad(0.06), steps=10, totalSteps=6000, repeat=1, scaling=4.0)

In [11]:
dataPath, ontimes, angles, speeds = eng.phiOntimeScan(speed=np.deg2rad(0.03), steps=20, totalSteps=12000, repeat=1, scaling=4.0, tolerance=np.deg2rad(3.0))

2020-08-21 15:01:53.204Z butler           30 butler.py:58 newRun: /data/MCS/20200821_016
2020-08-21 15:01:53.208Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-21 15:01:53.208Z camera           20 asrdCam.py:17 asrd...


Camera close!


2020-08-21 15:01:53.885Z engineer         20 engineer.py:993 Move phi to CCW home and reset motor scaling
2020-08-21 15:01:53.888Z cobraCoach       20 cobraCoach.py:662 home cobras: theta=0, phi=-5000


Camera close!
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103149829
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103118685
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 15:02:18.461Z engineer         20 engineer.py:1003 === Forward phi on-time scan #1 ===


...

2020-08-21 15:27:47.468Z cobraCoach       30 cobraCoach.py:362 Cobra#51 is too close to center


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 15:27:50.340Z cobraCoach       30 cobraCoach.py:362 Cobra#51 is too close to center
2020-08-21 15:27:50.341Z engineer         20 engineer.py:1089 done: [24 50], 1 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 15:27:58.834Z engineer         20 engineer.py:1089 done: [2], 0 left
2020-08-21 15:27:58.834Z engineer         20 engineer.py:1091 all cobras reach CCW limits


In [28]:
# swtich to theta mode
eng.setThetaMode()

In [32]:
# generate a quick motormap and set geometry
eng.makeThetaMotorMaps(f'{mod}_theta_quick.xml', steps=500, totalSteps=12000, repeat=1, fast=False)

2020-08-25 11:17:31.849Z butler           30 butler.py:58 newRun: /data/MCS/20200825_000
2020-08-25 11:17:31.852Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-25 11:17:31.853Z camera           20 asrdCam.py:17 asrd...


Camera close!


2020-08-25 11:17:32.554Z cobraCoach       20 cobraCoach.py:1076 theta home -10000 steps


Camera close!
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103149829
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103118685
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-25 11:18:10.068Z cobraCoach       20 cobraCoach.py:1088 1/1 theta forward to 500


...

getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-25 11:21:39.956Z cobraCoach       20 cobraCoach.py:1131 done: [15], 2 left
2020-08-25 11:21:39.956Z cobraCoach       20 cobraCoach.py:1122 1/1 theta backward to 12000


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-25 11:21:44.083Z cobraCoach       20 cobraCoach.py:1131 done: [25], 1 left
2020-08-25 11:21:44.083Z cobraCoach       30 cobraCoach.py:1137 1 did not reach theta CCW limit:
2020-08-25 11:21:44.084Z cobraCoach       30 cobraCoach.py:1142   Cobra(module=1 board=1 cobraNum=19 cobra=10: [143.03 140.66 132.6 ]
2020-08-25 11:21:44.084Z cobraCoach       20 cobraCoach.py:1145 1/1 theta reverse -10000 steps to limit
2020-08-25 11:22:09.500Z pfiDesign        20 pfiDesign.py:807 wrote pfiDesign file for 57 cobras and name=None to /data/MCS/20200825_000/output/SC09_theta_quick.xml


(PosixPath('/data/MCS/20200825_000'), array([], dtype=int64))

In [33]:
# set theta geometry from previous run
cc.setThetaGeometryFromRun('/data/MCS/20200825_000', onlyIfClear=False)

In [ ]:
# move theta to home position
cc.moveToHome(cc.goodCobras, thetaEnable=True, thetaCCW=False)

In [ ]:
# set current angle to zeros, if you know
cc.setCurrentAngles(cc.goodCobras, thetaAngles=0, phiAngles=np.pi/3)

In [ ]:
# theta on time search
eng.thetaOnTimeSearch(f'{mod}_theta_search.xml', iteration=5, repeat=1)

In [ ]:
# generate new motormaps
#eng.makeThetaMotorMaps(f'{mod}_theta800.xml', steps=800, totalSteps=12000, repeat=3)
#eng.makeThetaMotorMaps(f'{mod}_theta500.xml', steps=500, totalSteps=12000, repeat=3)
#eng.makeThetaMotorMaps(f'{mod}_theta200.xml', steps=200, totalSteps=12000, repeat=1)
#eng.makeThetaMotorMaps(f'{mod}_theta100.xml', steps=100, totalSteps=12000, repeat=3)
#eng.makeThetaMotorMaps(f'{mod}_theta50.xml', steps=50, totalSteps=12000, repeat=3)
eng.makeThetaMotorMaps(f'{mod}_theta.xml', steps=50, totalSteps=15000, repeat=3, fast=True)

In [ ]:
eng.makeThetaMotorMaps(f'{mod}_theta20.xml', steps=20, totalSteps=12000, repeat=3, force=True)
eng.makeThetaMotorMaps(f'{mod}_theta10.xml', steps=10, totalSteps=12000, repeat=3, force=True)

In [ ]:
eng.makeThetaMotorMaps(f'{mod}_theta5.xml', steps=5, totalSteps=12000, repeat=3, force=True)

In [ ]:
eng.makeThetaMotorMaps(f'{mod}_theta_test.xml', steps=100, totalSteps=12000, repeat=3, fast=True, thetaOnTime=thetaOt)

In [ ]:
eng.makeThetaMotorMaps(f'{mod}_theta_test.xml', steps=25, totalSteps=12000, repeat=3, fast=True, thetaOnTime=thetaOt)

In [ ]:
# reset motor scaling
cc.pfi.resetMotorScaling(cc.allCobras)

In [ ]:
# move by angles
dataPath, diffAngles, moves = eng.moveThetaAngles(cc.goodIdx, np.pi/2, tries=6)

In [ ]:
# convergence test
moves = eng.thetaConvergenceTest(cc.goodIdx, margin=15.0, runs=3, tries=8, fast=False, tolerance=0.1)

In [ ]:
moves = eng.thetaConvergenceTest2(cc.goodIdx, margin=15.0, runs=34, tries=10, threshold=0.2, tolerance=0.0015)

In [ ]:
# move theta arms upward in theta mode
eng.moveThetaAngles(cc.goodIdx, np.pi/2, local=False)

In [20]:
# theta on-time scan
dataPath, ontimes, angles, speeds = eng.thetaOntimeScan(speed=np.deg2rad(0.03), steps=20, totalSteps=20000, repeat=1, scaling=4.0, tolerance=np.deg2rad(3.0))

2020-08-21 15:55:47.874Z butler           30 butler.py:58 newRun: /data/MCS/20200821_021
2020-08-21 15:55:47.877Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-21 15:55:47.878Z camera           20 asrdCam.py:17 asrd...


Camera close!


2020-08-21 15:55:48.573Z engineer         20 engineer.py:1155 Move theta to CCW home and reset motor scaling
2020-08-21 15:55:48.576Z cobraCoach       20 cobraCoach.py:662 home cobras: theta=-10000, phi=0


Camera close!
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103149829
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103118685
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 15:56:25.822Z engineer         20 engineer.py:1165 === Forward theta on-time scan #1 ===


...

2020-08-21 17:00:00.013Z engineer         20 engineer.py:1251 done: [ 7 14], 2 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 17:00:11.031Z engineer         20 engineer.py:1251 done: [43], 1 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-21 17:00:25.667Z engineer         20 engineer.py:1251 done: [41], 0 left
2020-08-21 17:00:25.668Z engineer         20 engineer.py:1253 all cobras reach CCW limits


In [ ]:
# theta on-time scan
dataPath, ontimes, angles, speeds = eng.thetaOntimeScan(speed=np.deg2rad(0.06), steps=50, totalSteps=10000, repeat=2, scaling=4.0)

In [ ]:
# update cobra geometry
eng.updateXML(f'{mod}_new.xml', '/data/MCS/20200821_003', '/data/MCS/20200821_010')

In [35]:
# swtich to normal mode
eng.setNormalMode()

In [ ]:
# move to home position
cc.moveToHome(cc.goodCobras, thetaEnable=True, phiEnable=True, thetaCCW=False)
#cc.moveToHome([cc.allCobras[46]], thetaEnable=True, phiEnable=True, thetaCCW=True)

In [ ]:
# set current angle, if you know
cc.setCurrentAngles(cc.goodCobras, thetaAngles=0, phiAngles=0)

In [ ]:
# move by angles
dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, np.pi/2, np.pi/3, relative=False, local=True, tolerance=0.1, tries=6, homed=False, newDir=True)

In [ ]:
# move by positions, go home first and do safe movement
tries = 12
targets = cc.pfi.anglesToPositions(cc.allCobras, np.full(cc.nCobras, np.pi), np.full(cc.nCobras, np.pi/2))
dataPath, positions, moves = eng.movePositions(cc.goodIdx, targets[cc.goodIdx], tolerance=0.1, tries=tries, thetaMarginCCW=0.1, homed=True, newDir=True)

In [ ]:
# move by positions again
targets = cc.pfi.anglesToPositions(cc.allCobras, np.full(cc.nCobras, np.pi), np.full(cc.nCobras, np.pi/2))
dataPath, positions, moves = eng.movePositions(cc.goodIdx, targets[cc.goodIdx], tolerance=0.1, tries=8, thetaMarginCCW=0.1, homed=False, newDir=True)

In [ ]:
# 2D target convergence test
targets, moves = eng.convergenceTest(cc.goodIdx, runs=100, thetaMargin=np.deg2rad(15.0), phiMargin=np.deg2rad(25.0), tries=10, tolerance=0.1)

In [ ]:
targets, moves = eng.convergenceTest2(cc.goodIdx, runs=100, thetaMargin=np.deg2rad(20.0), phiMargin=np.deg2rad(20.0), tries=10, tolerance=0.1, thetaThreshold=0.2, phiThreshold=0.2)

2020-08-25 11:47:14.771Z butler           30 butler.py:58 newRun: /data/MCS/20200825_001
2020-08-25 11:47:14.775Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-25 11:47:14.775Z camera           20 asrdCam.py:17 asrd...


Camera close!


2020-08-25 11:47:15.475Z engineer         20 engineer.py:1290 === Run 1: Convergence test ===
2020-08-25 11:47:15.478Z engineer         20 engineer.py:1740 Move theta arms to angle=[161.92  70.46  38.07 171.12 267.81 141.22 292.46 195.42 100.52  44.81
 153.24 255.76 336.49 157.72  90.47 112.57 219.66 276.22 247.23 138.03
  87.5   42.85  84.43 174.97 247.29  63.05 102.02 294.36 345.47 176.84
 182.1  291.58  45.05 230.3  254.33 349.39  22.25 244.14 223.82 170.47
 285.8  360.58 294.78 101.32 356.68 119.56 139.52 238.79  89.08 120.5
 333.55 219.38  20.61  49.31 245.55 267.13 277.3 ] degree
2020-08-25 11:47:15.479Z engineer         20 engineer.py:1741 Move phi arms to angle=[ 87.47  31.65  38.8  152.5   40.06  23.96 121.39  46.42  29.79 104.85
  22.86  49.66  84.58  36.15  52.66  70.53  49.13  52.67  80.76  55.7
  39.16 140.1   55.66  47.84 108.56  45.05  52.49 104.02  37.69  32.4
 113.53  25.46  51.68  62.08  34.22  51.43 120.24  49.39  56.68  60.85
  37.93  50.99 102.34  41.5   55.47 141.

Camera close!
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103149829
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103118685
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-25 11:48:05.057Z cobraCoach       30 cobraCoach.py:387 Phi scale negative: Cobra#47, steps:21.0, angle:0.8141803335394896
2020-08-25 11:48:05.058Z engineer         20 engineer.py:1773 done: [1], 56 left


...

2020-08-25 13:43:38.305Z engineer         20 engineer.py:1773 done: [25], 2 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-25 13:43:41.630Z cobraCoach       30 cobraCoach.py:379 Theta scale negative: Cobra#15, steps:-4.0, angle:2.5559566750594174
2020-08-25 13:43:41.631Z engineer         20 engineer.py:1773 done: [14], 1 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-25 13:43:48.369Z engineer         20 engineer.py:1773 done: [15], 0 left
2020-08-25 13:43:48.369Z engineer         20 engineer.py:1775 all cobras are in positions
2020-08-25 13:43:48.370Z engineer         20 engineer.py:1290 === Run 96: Convergence test ===
2020-08-25 13:43:48.372Z engineer         20 engineer.py:1740 Move theta arms to angle=[160.85 173.84 129.12 328.3  182.82 169.17 303.72 311.57 346.63 322.78
 303.75 301.81 110.99 195.47 174.86 272.23  44.24  78.34 298.74 350.27
 177.7  257.42 184.99 122.59 219.39 290.35  50.17 110.46 278.38  86.65
 260.27  72.68 168.27 342.77 143.18 277.54 300.62 210.51 158.67  76.9
 131.75  35.63 338.21 347.94 320.04 134.52 251.45 352.4   36.12 175.49
 151.04 264.24 278.39 163.83 329.02 112.25 107.6 ] degree
2020-08-25 13:43:48.373Z engineer         20 engineer.py:1741 Move phi arms to angle=[ 56.2  132.42  33.63  54.05 134.6   22.47  51.39  68.62  41.    32.66
 136.48  34.4   42.98  96.23  53.95  59.4  121.39  28.91  27.26 149.49
  40.19 

In [ ]:
# save results in npy format
np.save('targets2', targets)
np.save('moves2', moves)

In [ ]:
# save results for convergence test in csv format
runs, nCobras, tries = moves.shape
mTargets = np.zeros(runs*nCobras, dtype=np.dtype(dict(names=['runs', 'cobra', 'x', 'y', 'theta', 'phi'], formats=['i4', 'i4', 'f4', 'f4', 'f4', 'f4'])))
mMoves = np.zeros(runs*nCobras*tries, dtype=np.dtype(dict(names=['runs', 'cobra', 'tries', 'x', 'y', 'theta', 'thetaSteps', 'thetaOntime', 'thetaFast', 'phi', 'phiSteps', 'phiOntime', 'phiFast'], formats=['i4', 'i4', 'i4', 'f4', 'f4', 'f4', 'i4', 'f4', 'i4', 'f4', 'i4', 'f4', 'i4'])))

for m in range(runs):
    mPositions = cc.pfi.anglesToPositions(cc.goodCobras, targets[m,:,0], targets[m,:,1])
    for n in range(nCobras):
        mTargets[m*nCobras+n] = m+1, cc.goodIdx[n]+1, mPositions[n].real, mPositions[n].imag, np.rad2deg(targets[m,n,0]), np.rad2deg(targets[m,n,1])
        for p in range(tries):
            mMoves[(m*nCobras+n)*tries+p] = m+1, cc.goodIdx[n]+1, p+1, moves[m,n,p]['position'].real, moves[m,n,p]['position'].imag, np.rad2deg(moves[m,n,p]['thetaAngle']), moves[m,n,p]['thetaSteps'], moves[m,n,p]['thetaOntime'], moves[m,n,p]['thetaFast'], np.rad2deg(moves[m,n,p]['phiAngle']), moves[m,n,p]['phiSteps'], moves[m,n,p]['phiOntime'], moves[m,n,p]['phiFast']

np.savetxt(f'targets2.csv', mTargets, fmt='%d %d %.2f %.2f %.2f %.2f', header='runs cobra x y theta phi')
np.savetxt(f'moves2.csv', mMoves, fmt='%d %d %d %.2f %.2f %.2f %d %.4f %d %.2f %d %.4f %d', header='runs cobra tries x y theta thetaSteps thetaOntime thetaFast phi phiSteps phiOntime phiFast')

In [27]:
# goto safe position
tht = np.empty(cc.nCobras)
tht[0::2] = np.pi*3/2
tht[1::2] = np.pi/2
#dataPath, thetas, phis, moves = eng.moveThetaPhi(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, newDir=True)
dataPath, thetas, phis, moves = eng.moveThetaPhi2(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, thetaThreshold=0.2, phiThreshold=0.2, newDir=True)

2020-08-24 15:27:01.344Z butler           30 butler.py:58 newRun: /data/MCS/20200824_000
2020-08-24 15:27:01.351Z fpgaIO           20 pfi.py:72 FPGA connection to 128.149.77.24
2020-08-24 15:27:01.352Z camera           20 asrdCam.py:17 asrd...


Camera close!


2020-08-24 15:27:02.040Z engineer         20 engineer.py:1740 Move theta arms to angle=[174.57   1.21 122.91  68.41 101.64 132.65 318.11 247.09 263.05  26.76
  70.5  258.05  99.56  98.01 146.27 348.67 205.5  306.1  344.28 285.46
  15.95 333.83  50.18 246.15 113.8    4.86 216.86 280.64  28.42 238.55
 218.74 265.73 267.79 186.48 244.45 132.9   23.23  42.85 230.9  149.08
 255.59  90.19  72.45  42.42  28.01  34.51 211.52 222.75 121.8   89.37
 275.58 124.51 189.6  242.97 117.08 119.3   95.15] degree
2020-08-24 15:27:02.041Z engineer         20 engineer.py:1741 Move phi arms to angle=[53.8  55.95 57.   59.2  58.95 59.07 52.88 57.3  56.88 57.77 56.14 53.15
 55.6  56.07 57.29 55.59 55.16 56.02 55.05 54.51 54.78 54.47 56.88 54.39
 59.34 61.1  56.7  64.76 59.5  63.11 56.83 58.71 61.97 58.12 58.27 63.84
 60.43 62.09 62.59 61.52 56.13 63.91 61.65 60.44 61.17 62.92 56.51 57.66
 64.37 54.08 62.06 59.69 58.55 58.93 65.64 61.56 62.13] degree
2020-08-24 15:27:02.042Z engineer         20 engineer.py:174

Camera close!
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103149829
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
IS_COLORMODE_MONOCHROME: 
	m_nColorMode: 		 34
	nBitsPerPixel: 		 10
	bytes_per_pixel: 		 2

Camera model:		 UI318xCP-M
Camera serial no.:	 4103118685
Maximum image self.width:	 2592
Maximum image self.height:	 2048

200
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-24 15:27:49.610Z pfi              40 pfi.py:847 Some theta angles are out of range


...

2020-08-24 15:28:07.440Z engineer         20 engineer.py:1773 done: [12 41 44], 2 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-24 15:28:09.447Z engineer         20 engineer.py:1773 done: [56], 1 left


getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)
getting image
(2048, 2592, 2)


2020-08-24 15:28:13.483Z engineer         20 engineer.py:1773 done: [42], 0 left
2020-08-24 15:28:13.484Z engineer         20 engineer.py:1775 all cobras are in positions


In [ ]:
cc.cobraInfo[47]

In [ ]:
cc.cobraInfo[47]

In [ ]:
cc.cobraInfo[47]

In [ ]:
cc.cobraInfo[47]

In [ ]:
cc.cobraInfo[47]

In [ ]:
cobras = [cc.allCobras[47]]
#cc.moveToHome(cobras, thetaEnable=True, phiEnable=True, thetaCCW=False)
#cc.moveDeltaAngles(cobras, thetaAngles=None, phiAngles=np.deg2rad(30), thetaFast=True, phiFast=True)
cc.moveDeltaAngles(cobras, thetaAngles=np.deg2rad(60), phiAngles=None, thetaFast=True, phiFast=True)
#dataPath, thetas, phis, moves = eng.moveThetaPhi2(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, thetaThreshold=0.2, phiThreshold=0.2, newDir=True)

In [ ]:
cc.pfi.moveThetaPhi(cobras, [np.deg2rad(60)], [0], [0.3740908], [0.5304951], True, True, doRun=False)

In [ ]:
dataPath, thetas, phis, moves = eng.moveThetaPhi2(cc.goodIdx, tht[cc.goodIdx], np.pi/3, relative=False, local=False, tolerance=0.1, tries=10, homed=False, thetaThreshold=0.2, phiThreshold=0.2, newDir=True)

In [ ]:
cc.showStatus()

In [ ]:
targets[0,0], moves[0,0], targets[0,35], moves[0,35], targets[0,37], moves[0,37], targets[0,53], moves[0,53]

In [ ]:
cc.moveInfo

In [ ]:
cc.pfi.calibModel.cobraStatus(31)

In [ ]:
cc.pfi.calibModel.setCobraStatus(31, brokenTheta=True)
cc.pfi.calibModel.createCalibrationFile('SC32_5.xml')

In [ ]:
np.rad2deg((cc.calibModel.tht1 - cc.calibModel.tht0 + np.pi) % (np.pi*2) - np.pi)

In [ ]:
cc.goodIdx

In [ ]:
moves[30,10:20,:]

In [ ]:
cc.setScaling(constantSpeedScaling=False)
targets, moves = eng.convergenceTest(cc.goodIdx, runs=100, thetaMargin=np.deg2rad(10.0), phiMargin=np.deg2rad(25.0), tries=20, tolerance=0.1)
cc.setScaling(constantSpeedScaling=True)
targets, moves = eng.convergenceTest(cc.goodIdx, runs=100, thetaMargin=np.deg2rad(10.0), phiMargin=np.deg2rad(25.0), tries=20, tolerance=0.1)

In [ ]:
ontimes[1,0]

In [ ]:
np.rad2deg(angles[1,:10])

In [ ]:
cc.moveInfo

In [ ]:
cc.phiInfo

In [ ]:
cc.moveSteps(cc.goodCobras, 0, -5000)

In [ ]:
cc.moveInfo, cc.phiInfo

In [ ]:
np.abs(cc.cobraInfo['position'] - cc.calibModel.centers)

In [ ]:
cc.cobraInfo

In [ ]:
cc.pfi.positionsToAngles(cc.allCobras, cc.cobraInfo['position'])[2][2,0] & 17

In [ ]:
cc.calibModel.L1, cc.calibModel.L2

In [ ]:
np.diff(np.rad2deg(angles[0,0])) / 250

In [ ]:
np.rad2deg(angles[0,0]), ontimes[0,0], np.rad2deg(speeds[0,0])

In [ ]:
np.rad2deg(speeds[0,0,:])

In [ ]:
ontimes[1,0,:]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
datadir = '/data/MCS/20200622_006'
angles = np.load(f'{datadir}/data/angles.npy')
ontimes = np.load(f'{datadir}/data/ontimes.npy')
parameters = np.load(f'{datadir}/data/parameters.npy')

In [ ]:
parameters

In [ ]:
idx = 15

data = ontimes[idx,0]*1000
nz = np.where(data == 0)[0]
if len(nz) > 0:
    upper = np.where(data == 0)[0][0] - 1
else:
    upper = len(data)
xdata = np.rad2deg(angles)[idx,0,:upper]

plt.figure(figsize=(20,20))
ax = plt.subplot(411)
plt.plot(xdata, data[:upper], 'o')
ax.set_title('On-time (forward)')

ax = plt.subplot(412)
data1 = np.rad2deg(speeds[idx,0])
plt.plot(xdata, data1[:upper], 'o')
ax.set_title('Step size (forward)')

data = ontimes[idx,1]*1000
nz = np.where(data == 0)[0]
if len(nz) > 0:
    upper = np.where(data == 0)[0][0]
else:
    upper = len(data)
xdata = np.rad2deg(angles)[idx,1,:upper]

ax = plt.subplot(413)
plt.plot(xdata, data[:upper], 'o')
ax.set_title('On-time (backward)')

ax = plt.subplot(414)
data1 = -np.rad2deg(speeds[idx,1])
plt.plot(xdata, data1[:upper], 'o')
ax.set_title('Step size (backward)')

plt.show()

In [ ]:
np.where(ontimes[0,0,:] == 0)[0][0]

In [ ]:
angles[1,1,:]

In [ ]:
np.rad2deg(np.diff(angles[0,0])) / 30

In [ ]:
np.diff(np.rad2deg(angles))[0,0]

In [ ]:
mod = 'SC09'

In [ ]:
ontimes[1,1,0,:]

In [ ]:
thetaOt = np.array([[0.0798, 0.0687, 0.0904, 0.0404, 0.0567, 0.0471, 0.0626, 0.0589,
        0.064 , 0.0645, 0.0464, 0.0447, 0.0563, 0.0372, 0.0441, 0.0613,
        0.0739, 0.0492, 0.1072, 0.052 , 0.0579, 0.0461, 0.0988, 0.0698,
        0.0546, 0.0648, 0.055 , 0.0458, 0.0519, 0.0563, 0.055 , 0.0492,
        0.0659, 0.0614, 0.0762, 0.0452, 0.0433, 0.0562, 0.0485, 0.0521,
        0.0502, 0.068 , 0.0561, 0.0669, 0.048 , 0.0575, 0.0658, 0.0436,
        0.0694, 0.0454, 0.0515, 0.0709, 0.0502, 0.0587, 0.0477, 0.1135,
        0.0494],
       [0.0808, 0.0551, 0.0812, 0.0442, 0.0597, 0.0537, 0.0623, 0.0588,
        0.0764, 0.0624, 0.0521, 0.0459, 0.062 , 0.0391, 0.0477, 0.0685,
        0.0714, 0.0505, 0.0956, 0.0412, 0.0574, 0.0521, 0.0926, 0.0766,
        0.0524, 0.0898, 0.0478, 0.0514, 0.0519, 0.0658, 0.0476, 0.0685,
        0.069 , 0.0807, 0.0702, 0.0489, 0.0406, 0.0579, 0.0554, 0.0511,
        0.0535, 0.0841, 0.056 , 0.0629, 0.0516, 0.0501, 0.05  , 0.0539,
        0.0667, 0.0459, 0.0564, 0.0907, 0.0559, 0.07  , 0.0496, 0.0758,
        0.0454]])

In [ ]:
phiOt = np.array([[0.0524, 0.0249, 0.0364, 0.0217, 0.0284, 0.0211, 0.0354, 0.029 ,
        0.0271, 0.0284, 0.0315, 0.02  , 0.028 , 0.0346, 0.0422, 0.0314,
        0.0313, 0.0327, 0.0484, 0.025 , 0.022 , 0.0247, 0.0258, 0.0454,
        0.0351, 0.0399, 0.0313, 0.0313, 0.0382, 0.039 , 0.024 , 0.0225,
        0.0528, 0.02  , 0.0481, 0.0303, 0.0209, 0.0347, 0.0237, 0.034 ,
        0.021 , 0.031 , 0.0301, 0.0314, 0.0335, 0.0286, 0.0229, 0.0238,
        0.0463, 0.0288, 0.0425, 0.0312, 0.0229, 0.026 , 0.0222, 0.0399,
        0.0211],
       [0.0394, 0.0277, 0.0376, 0.0228, 0.0261, 0.023 , 0.0319, 0.029 ,
        0.0269, 0.0282, 0.0323, 0.024 , 0.0285, 0.0364, 0.0516, 0.0326,
        0.032 , 0.0288, 0.0405, 0.0302, 0.021 , 0.0262, 0.0307, 0.0457,
        0.0344, 0.0359, 0.03  , 0.0356, 0.0383, 0.0403, 0.0222, 0.0217,
        0.051 , 0.0211, 0.0494, 0.0339, 0.02  , 0.033 , 0.0334, 0.04  ,
        0.0214, 0.0329, 0.0311, 0.0278, 0.026 , 0.0292, 0.021 , 0.0213,
        0.0412, 0.0321, 0.0332, 0.0309, 0.024 , 0.0273, 0.0212, 0.0332,
        0.0231]])

In [24]:
np.where(moves['position'][:,:,9] != 0)

(array([ 1,  1,  4,  4,  4,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 10,
        11, 11, 11, 12, 12, 13, 14, 14, 14, 15, 15, 15, 15, 17, 17, 18, 18,
        18, 18, 18, 18, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 22, 22,
        23, 24, 24, 24, 25, 25, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 31,
        31, 31, 31, 33, 33, 33, 33, 34, 34, 34, 34, 35, 35, 35, 36, 36, 36,
        37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 39, 40, 40, 40, 41, 42, 42,
        44, 44, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 48, 48, 48, 48, 50,
        50, 50, 50, 50, 51, 51, 52, 52, 53, 53, 53, 53, 53, 54, 54, 54, 54,
        54, 54, 54, 54, 54, 55, 55, 56, 56, 56, 56, 56, 56, 56, 56, 56, 57,
        58, 58, 59, 59, 60, 62, 62, 63, 63, 63, 63, 64, 64, 64, 64, 65, 66,
        66, 67, 67, 67, 68, 69, 69, 70, 71, 72, 72, 72, 72, 73, 73, 75, 76,
        77, 77, 77, 77, 77, 77, 77, 77, 77, 80, 80, 80, 82, 82, 82, 82, 83,
        84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89, 89, 89, 90, 90, 90,
        90, 

In [ ]:
moves[26], targets[26]

In [26]:
np.count_nonzero(moves[:,:,9]['position'], axis=0)

array([ 4,  8,  7,  8,  6,  4,  3,  2,  1,  1,  6,  3,  6,  5,  5,  9,  8,
        5, 13,  4,  4,  7,  5,  7,  3,  5,  4,  3,  7,  3,  5, 12,  8,  7,
        7,  4,  4,  1,  0,  1,  1,  1,  3,  3,  2,  1,  3,  4,  1,  0,  1,
        1,  3,  1,  3,  3,  2])

In [ ]:
np.deg2rad(15), np.deg2rad(180), np.deg2rad(345)

In [ ]:
eng.mmTheta[8,0,550:580], eng.mmTheta[8,1,300:340]

In [ ]:
np.where((cc.thetaInfo['cwHome'] - cc.thetaInfo['ccwHome']) % (np.pi*2) > 1.0)

In [ ]:
cc.thetaInfo[22], cc.thetaInfo[23]

In [ ]:
cc.thetaInfo[22]['cwHome'] = 3.1

In [ ]:
(cc.calibModel.tht1 - cc.calibModel.tht0)[22:24]

In [ ]:
dataPath

In [34]:
cc.calibModel.centers - cc.thetaInfo['center']

array([0.38002941+1.30144878e+00j, 0.37449376+1.29969714e+00j,
       0.34468667+1.33930592e+00j, 0.41949554+1.36650946e+00j,
       0.3066881 +1.31517152e+00j, 0.40094732+1.29471595e+00j,
       0.39144668+1.26963689e+00j, 0.33008769+1.26243867e+00j,
       0.41302397+1.24602901e+00j, 0.35871075+1.24968314e+00j,
       0.37191573+1.25060530e+00j, 0.34623334+1.28862074e+00j,
       0.38109565+1.23206868e+00j, 0.36119296+1.25210834e+00j,
       0.34553228+1.23243980e+00j, 0.3778119 +1.19903809e+00j,
       0.39469697+1.18777407e+00j, 0.37639042+1.20781253e+00j,
       0.41329253+1.20773479e+00j, 0.37840993+1.19480494e+00j,
       0.38275195+1.15585232e+00j, 0.38016175+1.18662969e+00j,
       0.39453249+1.16601380e+00j, 0.42862599+1.17000911e+00j,
       0.3641739 +1.12537246e+00j, 0.36828194+1.11985764e+00j,
       0.44860743+1.18204817e+00j, 0.4026879 +1.14534343e+00j,
       0.52890269+1.12289048e+00j, 0.20693663+8.65799461e-02j,
       0.18984797+3.43459128e-02j, 0.22958921+3.8196145

In [31]:
cc.calibModel.centers - cc.thetaInfo['center']

array([0.32680675+0.28631206j, 0.32078282+0.12324451j,
       0.23531167+0.19984059j, 0.30230804+0.21691229j,
       0.23539904+0.17668275j, 0.23932622+0.12686927j,
       0.2581459 +0.19193913j, 0.23243144+0.15605439j,
       0.25140287+0.11095821j, 0.253242  +0.11021781j,
       0.27328292+0.16289778j, 0.2834892 +0.13847426j,
       0.30199409+0.15662922j, 0.30113437+0.17026019j,
       0.2591065 +0.09602622j, 0.29529237+0.08459717j,
       0.33927705+0.10879458j, 0.28459354+0.07731937j,
       0.30635893+0.10452434j, 0.28710134+0.10026148j,
       0.29681445+0.05557155j, 0.29324769+0.06089726j,
       0.27612429+0.06500062j, 0.30655567+0.05355403j,
       0.2626114 +0.0072084j , 0.27770577+0.0425261j ,
       0.36975001+0.04740461j, 0.28208243+0.0058781j ,
       0.3450648 +0.0052147j , 0.23171691-1.19119105j,
       0.2098675 -1.24550028j, 0.24826597-1.25446743j,
       0.24836119-1.23414872j, 0.23589947-1.23252945j,
       0.24481381-1.3209983j , 0.14751656-1.31427004j,
       0.1

In [7]:
moves[3]

array([(4402.44644098+1001.1336226j , 1.0232083, -2104, 0.051,  True, 1.0948124 ,  327, 0.031,  True),
       (4398.50659509+1001.18389828j, 1.1563889,   337, 0.024,  True, 0.94586253, -103, 0.016,  True),
       (4400.42128378+1006.07896006j, 1.1928524,    74, 0.027,  True, 1.1241108 ,  155, 0.019,  True),
       (4398.54825067+1002.89483071j, 1.197289 ,     2, 0.027,  True, 0.98270696, -151, 0.013,  True),
       (4399.83098628+1004.89996408j, 1.1931425,    -5, 0.022,  True, 1.0742795 ,   88, 0.016,  True),
       (4399.08229254+1003.50417567j, 1.1917536,     1, 0.027,  True, 1.0152887 ,  -68, 0.012,  True),
       (4399.34549983+1004.14294793j, 1.1957339,     4, 0.027,  True, 1.0391967 ,   31, 0.014,  True),
       (4399.28361022+1003.95714836j, 1.1941371,    -2, 0.022,  True, 1.0327262 ,  -10, 0.011,  True),
       (   0.           +0.j        , 0.       ,     0, 0.   , False, 0.        ,    0, 0.   , False),
       (   0.           +0.j        , 0.       ,     0, 0.   , False, 0. 